# Populate the DB with CSV dataset

## Modules needed and config

Make sure you have a valid .env file to load the correct MONGO_URL

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd

from pymongo import MongoClient
client = MongoClient(os.getenv("MONGO_URL"))

from datetime import datetime

## Import Dataset

In [ ]:
df_lter = pd.read_csv("./source/penguins_lter.csv")

## Setup MongoDB client and collection for data

In [ ]:
db = client["palmer-penguins"]
collection_kpl = db["kaggle-penguins-lter"]

## Dataset cleaning and formatting

In [ ]:
df_lter["Date Egg"]=df_lter["Date Egg"].apply(lambda e: datetime.strptime(e,'%m/%d/%y'))
df_lter = df_lter.drop("Comments", 1)
df_lter = df_lter.rename(columns=lambda x: x.split(
    "(")[0].strip().replace(" ", "_").lower())
df_lter["clutch_completion"] = df_lter["clutch_completion"].apply(
    lambda x: True if x == "Yes" else False)

## Transform DataFrame to MongoDB documents and insert
Output is the documents inserted on the database

In [ ]:
documents = df_lter.to_dict("records")
collection_kpl.drop()
result = collection_kpl.insert_many(documents)
len(result.inserted_ids)
client.close()
